In [ ]:
import numpy as np
import open3d as o3d

#### Read base

In [ ]:
base_filename = 'base'
base_pcd = o3d.io.read_point_cloud(f'../output/pcd/{base_filename}.ply')

#### Read source

In [ ]:
source_filename = 'left_20250917_154535_7'
source_pcd = o3d.io.read_point_cloud(f'../output/pcd/anomalies/{source_filename}.ply')

#### Get base plane

In [ ]:
# Fit plane to base point cloud
plane_model, inliers = base_pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)

# Extract plane coefficients [a, b, c, d] where ax + by + cz + d = 0
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.3f}x + {b:.3f}y + {c:.3f}z + {d:.3f} = 0")

# Get plane normal vector
plane_normal = np.array([a, b, c])
print(f"Plane normal: {plane_normal}")

# Visualize the plane fit
inlier_cloud = base_pcd.select_by_index(inliers)
outlier_cloud = base_pcd.select_by_index(inliers, invert=True)

# Color the inliers and outliers differently
inlier_cloud.paint_uniform_color([1.0, 0, 0])  # Red for plane
outlier_cloud.paint_uniform_color([0, 0, 1.0])  # Blue for outliers

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

#### Transform

In [ ]:
# Create transformation to align plane with XY plane
# Target normal is Z-axis [0, 0, 1]
target_normal = np.array([0, 0, 1])

# Normalize the plane normal
plane_normal_normalized = plane_normal / np.linalg.norm(plane_normal)

# Calculate rotation matrix using Rodrigues' rotation formula
# If normals are already aligned, use identity matrix
if np.allclose(plane_normal_normalized, target_normal):
    rotation_matrix = np.eye(3)
elif np.allclose(plane_normal_normalized, -target_normal):
    # If normals are opposite, rotate 180 degrees around any perpendicular axis
    rotation_matrix = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]])
else:
    # Calculate rotation axis (cross product)
    rotation_axis = np.cross(plane_normal_normalized, target_normal)
    rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
    
    # Calculate rotation angle
    cos_angle = np.dot(plane_normal_normalized, target_normal)
    angle = np.arccos(np.clip(cos_angle, -1, 1))
    
    # Create rotation matrix using Rodrigues' formula
    K = np.array([[0, -rotation_axis[2], rotation_axis[1]],
                  [rotation_axis[2], 0, -rotation_axis[0]],
                  [-rotation_axis[1], rotation_axis[0], 0]])
    
    rotation_matrix = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)

# Create 4x4 transformation matrix
transformation = np.eye(4)
transformation[:3, :3] = rotation_matrix

print(f"Rotation matrix:\n{rotation_matrix}")
print(f"Transformation matrix:\n{transformation}")

# Apply transformation to both point clouds
base_pcd_transformed = base_pcd.transform(transformation)
source_pcd_transformed = source_pcd.transform(transformation)

# Verify the transformation by checking the new plane normal
plane_model_new, _ = base_pcd_transformed.segment_plane(distance_threshold=0.01,
                                                        ransac_n=3,
                                                        num_iterations=1000)
[a_new, b_new, c_new, d_new] = plane_model_new
new_normal = np.array([a_new, b_new, c_new])
print(f"New plane normal: {new_normal}")
print(f"Should be close to [0, 0, ±1]: {np.allclose(np.abs(new_normal), [0, 0, 1], atol=0.1)}")

#### Save Transformation matrix

In [ ]:
np.save('../camera_configs/transformation_matrix.npy', transformation)